### UK TRQ Quality Assurance Notebook. 

#### The following notebook is a qualtiy assurance script for the UK TRQ analysis. 

The note book is split into the following parts:

**Section 1: TRQ data:**

1. Data cleaning
2. Aggregate data and calculate utilisation rates
3. Compare outputs

**Section 2: Filled quotas:**

1. Convert data to data format and filter for filled quotas
2. Filled quota calculations for average time taken
3. QA function and output comparison

**Section 3: TRQ sector data:**

1. Automate sector classification and assign sector per quota
2. Aggregate data to quota and sector level - using def fun in section 1. 
2. Update def function for sector data and save outputs for excel comparison




****

# Section 1 TRQ data

### **Part 1: data cleaning:**

Data uploads and clean column strings:

In [1]:
# upload data:

import pandas as pd
import numpy as np

!pip install openpyxl

# chnage from scientific notation 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

uk_trqs = pd.read_excel('../rsp_uk_trq_qa/data/uk_trq_data.xlsx',dtype={'Quota Number': str}) 
uk_trqs2 = pd.read_excel('../rsp_uk_trq_qa/data/uk_license_trq_input.xlsx',dtype={'Quota number': str}) 
eu_trqs = pd.read_excel('../rsp_uk_trq_qa/data/eu_trq_data.xlsx',dtype={'Order_Number': str}) 


print(uk_trqs.shape, uk_trqs2.shape, eu_trqs.shape)

Looking in indexes: https://s3-eu-west-2.amazonaws.com/mirrors.notebook.uktrade.io/pypi/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 KB 220.2 MB/s eta 0:00:00
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.
(1067, 14) (95, 17) (12445, 26)


In [2]:
# upload input files

uk_trq_country_input = pd.read_excel('../rsp_uk_trq_qa/data/uk_trq_country_input.xlsx')
hs_dsc = pd.read_excel('../rsp_uk_trq_qa/data/hs_descriptions.xlsx')
country_names = pd.read_excel('../rsp_uk_trq_qa/data/country_names.xlsx')
eu_trq_data_input = pd.read_excel('../rsp_uk_trq_qa/data/eu_trq_data_input.xlsx')
eu_trq_sector_manual_input = pd.read_excel('../rsp_uk_trq_qa/data/eu_trq_sector_manual_input.xlsx')
trq_quota_unit = pd.read_excel('../rsp_uk_trq_qa/data/trq_quota_unit.xlsx')

print(uk_trq_country_input.shape, hs_dsc.shape, country_names.shape, eu_trq_data_input.shape, eu_trq_sector_manual_input.shape,trq_quota_unit.shape)

(70, 4) (98, 3) (225, 4) (114, 4) (249, 11) (6, 3)


In [3]:
def  cleanCols(df): 
    df.columns = df.columns.str.lower().str.replace(" ","_")
    return(df)

In [4]:
# clean names of dataframes:

uk_trqs = cleanCols(uk_trqs)
uk_trqs2 = cleanCols(uk_trqs2)
eu_trqs = cleanCols(eu_trqs)
uk_trq_country_input = cleanCols(uk_trq_country_input)
hs_dsc = cleanCols(hs_dsc)
country_names = cleanCols(country_names)
eu_trq_data_input = cleanCols(eu_trq_data_input)
eu_trq_sector_manual_input = cleanCols(eu_trq_sector_manual_input)
trq_quota_unit = cleanCols(trq_quota_unit)

In [5]:
# clean and combine UK TRQ data:
print(uk_trqs.dtypes)

quota_number                  object
quota_product_description     object
year                           int64
quota_origin                  object
validity_start_date           object
validity_end_date             object
quota_volume                 float64
remaining_balance            float64
quota_fill_rate              float64
quota_unit                    object
commodity_codes               object
quota_status                  object
last_allocation_date          object
trq_type                      object
dtype: object


In [6]:
# select data for non-license quotas:

uk = uk_trqs.loc[:,["quota_number",
                   "quota_product_description",
                 "year",
                 "quota_origin",
                 "quota_volume",
                 "remaining_balance",
                 "quota_unit",
                 "commodity_codes"]
                ]

# rename

uk.rename(columns = {'quota_product_description':'quota_description', 'year':'quota_year'}, inplace = True)
uk.head()

# filter for quota year = 2021

uk = uk.loc[uk["quota_year"] == 2021]

# match country/region names for combining with license data and final aggregated outputs:

uk = pd.merge(uk, uk_trq_country_input, left_on = "quota_origin", right_on = "country_name", how = "left")

# filter away NA (i.e. non match)

uk = uk[uk['region'].notnull()]

uknan = uk[uk['region'].isnull()] # NaN df for reference. 

uk.head()

,quota_number,quota_description,quota_year,quota_origin,quota_volume,remaining_balance,quota_unit,commodity_codes,iso,country_name,grouping,region
67,050146,Live bovine animals,2021,Switzerland,247.00000,247.00000,Number of items (p/st),0102294100 ; 0102294900 ; 0102295100 ; 0102295...,CH,Switzerland,Switzerland,Wider Europe & Middle East
72,050155,"Meat and edible offal, of duch, fresh or forzen",2021,Israel,76000.00000,76000.00000,Kilogram (kg),0207423000 ; 0207428000 ; 0207441000 ; 0207442...,IL,Israel,Israel,Wider Europe & Middle East
78,050202,"Potatoes, fresh or chilled, other than seed",2021,Turkey,417000.00000,322220.00000,Kilogram (kg),0701900000,TR,Turkey,Turkey,Eastern Europe
79,050204,Fresh table grapes,2021,Turkey,58000.00000,58000.00000,Kilogram (kg),0806101090,TR,Turkey,Turkey,Eastern Europe
80,050204,Fresh table grapes,2021,Turkey,58000.00000,0.00000,Kilogram (kg),0806101090,TR,Turkey,Turkey,Eastern Europe


In [7]:
# convert quota unit measurments for consistency i.e. Tonnes to be in Kilograms, HectoLitres to be in Litres etc. 
# convert Tonnes to Kilograms. 
print(pd.unique(uk["quota_unit"]))

['Number of items (p/st)' 'Kilogram (kg)' 'Litre (l)' 'Hectolitre (hl)'
 'Tonne (1,000 kg)' 'Square metre (m2)' 'Number of pairs (pa)'
 'Litre pure (100%) alcohol (l alc. 100%)']


In [8]:
uk["tonne_flag"] = np.where(uk["quota_unit"].str.contains("Tonne"), True,False)
uk["quota_unit"] = np.where(uk["tonne_flag"] == True, "Kilogram (kg)", uk["quota_unit"])

In [9]:
uk["quota_volume"] = np.where(uk["tonne_flag"] == True,uk["quota_volume"]*1000,uk["quota_volume"])
uk["remaining_balance"] = np.where(uk["tonne_flag"] == True,uk["remaining_balance"]*1000,uk["remaining_balance"])

In [10]:
# convert HL to L:
uk["hl_flag"] = np.where(uk["quota_unit"].str.contains("Hecto"), True,False)
uk["quota_unit"] = np.where(uk["hl_flag"] == True, "Litre (l)", uk["quota_unit"])
uk["quota_volume"] = np.where(uk["hl_flag"] == True,uk["quota_volume"]*100,uk["quota_volume"])
uk["remaining_balance"] = np.where(uk["hl_flag"] == True,uk["remaining_balance"]*100,uk["remaining_balance"])


In [11]:
# match final quota unit df. This is to combine a common name across the EU and UK data to combine and aggregate. 

uk = pd.merge(uk, trq_quota_unit[["uk_quota_unit","quota_unit_final"]], left_on = "quota_unit", right_on = "uk_quota_unit", how = "left")
uk.head()

,quota_number,quota_description,quota_year,quota_origin,quota_volume,remaining_balance,quota_unit,commodity_codes,iso,country_name,grouping,region,tonne_flag,hl_flag,uk_quota_unit,quota_unit_final
0,050146,Live bovine animals,2021,Switzerland,247.00000,247.00000,Number of items (p/st),0102294100 ; 0102294900 ; 0102295100 ; 0102295...,CH,Switzerland,Switzerland,Wider Europe & Middle East,False,False,Number of items (p/st),Number of items
1,050155,"Meat and edible offal, of duch, fresh or forzen",2021,Israel,76000.00000,76000.00000,Kilogram (kg),0207423000 ; 0207428000 ; 0207441000 ; 0207442...,IL,Israel,Israel,Wider Europe & Middle East,False,False,Kilogram (kg),Kilograms
2,050202,"Potatoes, fresh or chilled, other than seed",2021,Turkey,417000.00000,322220.00000,Kilogram (kg),0701900000,TR,Turkey,Turkey,Eastern Europe,False,False,Kilogram (kg),Kilograms
3,050204,Fresh table grapes,2021,Turkey,58000.00000,58000.00000,Kilogram (kg),0806101090,TR,Turkey,Turkey,Eastern Europe,False,False,Kilogram (kg),Kilograms
4,050204,Fresh table grapes,2021,Turkey,58000.00000,0.00000,Kilogram (kg),0806101090,TR,Turkey,Turkey,Eastern Europe,False,False,Kilogram (kg),Kilograms


In [12]:
# remove columns:
uk = uk.drop(["tonne_flag","hl_flag","uk_quota_unit"], 1)
#uk.dtypes

In [13]:
# filter licnese quota data

uk2 = uk_trqs2.loc[uk_trqs2["year"] == 2021]
uk2.shape

(95, 17)

In [14]:
uk2 = uk2[uk2["rsp_region"].notnull()]
uk2.dtypes

quota_number                object
quota_description           object
year                         int64
quota_origin                object
rsp_region                  object
quota_period                object
quota_unit                  object
commodity_codes             object
trq_type                    object
2021_quantity_available      int64
2021_quantity_remaining    float64
2021_usage                 float64
2021_fill_rate             float64
unnamed:_13                float64
unnamed:_14                float64
unnamed:_15                float64
unnamed:_16                float64
dtype: object

In [15]:
# match country input file for consitent naming across dfs
uk2 = pd.merge(uk2, uk_trq_country_input, left_on = "quota_origin", right_on = "country_name", how = "left")
uk2.dtypes

quota_number                object
quota_description           object
year                         int64
quota_origin                object
rsp_region                  object
quota_period                object
quota_unit                  object
commodity_codes             object
trq_type                    object
2021_quantity_available      int64
2021_quantity_remaining    float64
2021_usage                 float64
2021_fill_rate             float64
unnamed:_13                float64
unnamed:_14                float64
unnamed:_15                float64
unnamed:_16                float64
iso                         object
country_name                object
grouping                    object
region                      object
dtype: object

In [16]:
# join quota unit input for consistent quota unit names across dfs
uk2 = pd.merge(uk2, trq_quota_unit[["uk_quota_unit","quota_unit_final"]], left_on = "quota_unit", right_on = "uk_quota_unit", how = "left")

In [17]:
uk2.dtypes

quota_number                object
quota_description           object
year                         int64
quota_origin                object
rsp_region                  object
quota_period                object
quota_unit                  object
commodity_codes             object
trq_type                    object
2021_quantity_available      int64
2021_quantity_remaining    float64
2021_usage                 float64
2021_fill_rate             float64
unnamed:_13                float64
unnamed:_14                float64
unnamed:_15                float64
unnamed:_16                float64
iso                         object
country_name                object
grouping                    object
region                      object
uk_quota_unit               object
quota_unit_final            object
dtype: object

In [18]:
uk2 = uk2.loc[:,["quota_number",
                "quota_description",
                 "year",
                 "quota_origin",
                 "2021_quantity_available",
                 "2021_quantity_remaining",
                 "quota_unit",
                 "commodity_codes",
                     "iso",
                     "country_name",
                     "grouping",
                     "region",
                     "quota_unit_final"]
                ]

uk2.rename(columns = {"2021_quantity_available":"quota_volume","2021_quantity_remaining":"remaining_balance", 'year':'quota_year'}, inplace = True)

In [19]:
# combine uk dfs together

uk3 = pd.concat([uk,uk2])
uk3.shape

(810, 13)

In [20]:
# eu data - match country input for consistent grouping/region names and filter

eu = eu_trqs.loc[eu_trqs["year"] == 2020]

eu = pd.merge(eu, eu_trq_data_input, on = "origin", how = "left")

eu = eu[eu["region"].notnull()] # filter Non region match

# rename order number:

eu.rename(columns = {'order_number':'quota_number', 'year':'quota_year'}, inplace = True)

In [21]:
# match quota unit for consistency across dfs
eu = pd.merge(eu, trq_quota_unit[["eu_quota_unit","quota_unit_final"]], left_on = "unit", right_on = "eu_quota_unit", how = "left")

In [22]:
eu.dtypes

quota_number                         object
origin                               object
description                          object
quota_year                            int64
application_period_starts    datetime64[ns]
application_period_ends      datetime64[ns]
volume                              float64
eu_usage                            float64
unit                                 object
last_import_date             datetime64[ns]
report_date                  datetime64[ns]
live                                float64
adj_ap_start                 datetime64[ns]
adj_ap_end                   datetime64[ns]
source                               object
cn8_codes                            object
cn10_codes                           object
inquota_rate                         object
regulations                          object
quota_type                           object
eu_usage_source                      object
origin_code_x                        object
country_exc                     

In [23]:
eu_trq_data_input

,origin,origin_code,grouping,region
0,ACP,"AO, AG, BB, BZ, BJ, BW, BF, BI, CM, CV, CF, TD...",ACP,NaN
1,ACP-Cariforum,"AO, AG, BB, BZ, BJ, BW, BF, BI, CM, CV, CF, TD...",ACP-Cariforum,NaN
2,Albania,AL,Albania,Eastern Europe
3,Algeria,DZ,Algeria,NaN
4,Argentina,AR,Argentina,NaN
...,...,...,...,...
109,WTO countries,E.O.,ERGA OMNES,NaN
110,"XK, MK, RS, ME, XW, AL, HR, BA","XK, MK, RS, ME, XW, AL, HR, BA","XK, MK, RS, ME, XW, AL, HR, BA",NaN
111,"XK, XV, RS, ME, BA","XK, XV, RS, ME, BA","XK, XV, RS, ME, BA",NaN
112,"XN, 2300","AR, BD, BR, SV, GT, HN, IN, ID, LA, PK, LK, TH","XN, 2300",NaN


### **2. Data aggregations and utilisation calculations**

Both UK and EU data need to be grouped by quota origin, grouping and region to produce 3 seperate outputs. 

Grouping is combined with quota unit as TRQs need to be outputted broken down by quota unit as they can't be mixed together (i.e. KG with Litre) as they highlight different measurments. 

When the grouped quota allocation is calculated the full allocaiton utilisation and rates need calculating. 

**UK data aggregation**

In [24]:
uk_agg_origin = uk3.groupby(["quota_origin","quota_unit_final"], as_index = False).agg({"quota_volume":"sum", 
                                                                                        "remaining_balance":"sum",
                                                                                       "quota_number":"count"})
uk_agg_origin["source"] = "UK" 

# calculate quota allocation usage and utilisation rate
uk_agg_origin["usage"] = uk_agg_origin["quota_volume"] - uk_agg_origin["remaining_balance"]
uk_agg_origin["fill_rate"] = uk_agg_origin["usage"] / uk_agg_origin["quota_volume"]

# remove remaining balance as not needed for outputs

#uk_agg_origin = uk_agg_origin.drop("remaining_balance",1)

# relocate columns (one option)
#col = uk_agg_origin[["usage","fill_rate"]]
#uk_agg_origin.drop(labels=[['usage',"fill_rate"]], axis=1, inplace = True)
#uk_agg_origin.insert(3,"usage",col)

# small df so can more easily name column order:

uk_agg_origin = uk_agg_origin[["quota_origin","quota_unit_final","quota_volume","usage","fill_rate","quota_number","source"]]

uk_agg_origin

,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Albania,Kilograms,384000.00000,0.00000,0.00000,7,UK
1,Albania,Litres,163400.00000,0.00000,0.00000,3,UK
2,Canada,Kilograms,149769000.00000,5861431.21000,0.03914,44,UK
3,Canada,Number of items,3152000.00000,55707.00000,0.01767,20,UK
4,Canada,Square metre (m2),13222000.00000,0.00000,0.00000,8,UK
...,...,...,...,...,...,...,...
61,Tunisia,Kilograms,9615400.00000,45300.00000,0.00471,11,UK
62,Tunisia,Litres,639700.00000,0.00000,0.00000,2,UK
63,Turkey,Kilograms,446451000.00000,280871448.63300,0.62912,64,UK
64,Ukraine,Kilograms,417412000.00000,65218829.00000,0.15625,46,UK


In [25]:
uk_agg_grouping = uk3.groupby(["grouping","quota_unit_final"], as_index = False).agg({"quota_volume":"sum", 
                                                                                      "remaining_balance":"sum",
                                                                                     "quota_number":"count"})
uk_agg_grouping["source"] = "UK" 

# calculate quota allocation usage and utilisation rate
uk_agg_grouping["usage"] = uk_agg_grouping["quota_volume"] - uk_agg_grouping["remaining_balance"]
uk_agg_grouping["fill_rate"] = uk_agg_grouping["usage"] / uk_agg_grouping["quota_volume"]

# remove remaining balance as not needed for outputs

#uk_agg_grouping = uk_agg_grouping.drop("remaining_balance",1)

# small df so can more easily name column order:

uk_agg_grouping = uk_agg_grouping[["grouping","quota_unit_final","quota_volume","usage","fill_rate","quota_number","source"]]

uk_agg_grouping.head()

,grouping,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Albania,Kilograms,384000.00000,0.00000,0.00000,7,UK
1,Albania,Litres,163400.00000,0.00000,0.00000,3,UK
2,Andean countries,Kilograms,101918000.00000,2602643.31200,0.02554,112,UK
3,Andean countries,Litre pure (100%) alcohol,45100.00000,0.00000,0.00000,2,UK
4,Andean countries,Litres,1700.00000,0.00000,0.00000,1,UK


In [26]:
uk_agg_region = uk3.groupby(["region","quota_unit_final"], as_index = False).agg({"quota_volume":"sum", 
                                                                                  "remaining_balance":"sum",
                                                                                 "quota_number":"count"})
uk_agg_region["source"] = "UK" 

uk_agg_region["usage"] = uk_agg_region["quota_volume"] - uk_agg_region["remaining_balance"]
uk_agg_region["fill_rate"] = uk_agg_region["usage"] / uk_agg_region["quota_volume"]

# remove remaining balance as not needed for outputs

#uk_agg_region = uk_agg_region.drop("remaining_balance",1)

# small df so can more easily name column order:

uk_agg_region = uk_agg_region[["region","quota_unit_final","quota_volume","usage","fill_rate","quota_number","source"]]

uk_agg_region

,region,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Africa,Kilograms,314629400.00000,185740879.04000,0.59035,60,UK
1,Africa,Litres,72122600.00000,62438545.17600,0.86573,4,UK
2,Asia Pacific,Kilograms,237435000.00000,110682361.00000,0.46616,46,UK
3,Asia Pacific,Square metre (m2),1068320.00000,0.00000,0.00000,1,UK
4,Eastern Europe,Kilograms,904875906.00000,346090277.63300,0.38247,138,UK
5,Eastern Europe,Litres,8085726.30000,234283.80000,0.02897,11,UK
6,The Americas,Kilograms,538510273.00000,71164672.66500,0.13215,237,UK
7,The Americas,Litre pure (100%) alcohol,196400.00000,136676.00000,0.69591,4,UK
8,The Americas,Litres,1700.00000,0.00000,0.00000,1,UK
9,The Americas,Number of items,23468686.00000,55707.00000,0.00237,139,UK


In [27]:
# alternative way to drop columns to not repeat lines:

for set_ in (uk_agg_origin, uk_agg_grouping, uk_agg_region):
    set_.drop("quota_number", axis=1, inplace=True)

In [28]:
print(uk_agg_origin.shape,uk_agg_grouping.shape,uk_agg_region.shape)

(66, 6) (46, 6) (15, 6)


**EU data aggregation**

In [29]:
eu_agg_origin = eu.groupby(["origin","quota_unit_final"], as_index = False).agg({"volume":"sum", 
                                                                                  "eu_usage":"sum",
                                                                                  "quota_number":"count"})
eu_agg_origin["source"] = "EU" 

# rename origin and usage to combine with UK data:

eu_agg_origin.rename(columns = {'origin':'quota_origin', 'eu_usage':'usage', "volume":"quota_volume"}, inplace = True)

# calculate utilisaiton (fill rate)

eu_agg_origin["fill_rate"] = eu_agg_origin["usage"] / eu_agg_origin["quota_volume"]

col = eu_agg_origin["fill_rate"]
eu_agg_origin.drop(labels=["fill_rate"], axis=1, inplace = True)
eu_agg_origin.insert(4,"fill_rate",col)

eu_agg_origin

,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,7,EU
1,Albania,Litres,700000.00000,4038.00000,0.00577,2,EU
2,Canada,Kilograms,370141000.00000,7833365.20700,0.02116,41,EU
3,Canada,Number of items,3203875.50000,199500.40000,0.06227,20,EU
4,Canada,Square metre (m2),16191000.00000,10274.63900,0.00063,9,EU
...,...,...,...,...,...,...,...
57,Tunisia,Kilograms,126299000.00000,68455753.00000,0.54201,11,EU
58,Tunisia,Litres,23520000.00000,15818.00000,0.00067,2,EU
59,Turkey,Kilograms,4045301876.00400,1799497888.22100,0.44484,68,EU
60,Ukraine,Kilograms,6716435736.75000,3154710277.55700,0.46970,77,EU


In [30]:
eu_agg_origin = eu.groupby(["origin","quota_unit_final"], as_index = False).agg({"volume":"sum", 
                                                                                  "eu_usage":"sum",
                                                                                  "quota_number":"count"})
eu_agg_origin["source"] = "EU" 

# rename origin and usage to combine with UK data:

eu_agg_origin.rename(columns = {'origin':'quota_origin', 'eu_usage':'usage', "volume":"quota_volume"}, inplace = True)

# calculate utilisaiton (fill rate)

eu_agg_origin["fill_rate"] = eu_agg_origin["usage"] / eu_agg_origin["quota_volume"]

col = eu_agg_origin["fill_rate"]
eu_agg_origin.drop(labels=["fill_rate"], axis=1, inplace = True)
eu_agg_origin.insert(4,"fill_rate",col)

eu_agg_origin

,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,7,EU
1,Albania,Litres,700000.00000,4038.00000,0.00577,2,EU
2,Canada,Kilograms,370141000.00000,7833365.20700,0.02116,41,EU
3,Canada,Number of items,3203875.50000,199500.40000,0.06227,20,EU
4,Canada,Square metre (m2),16191000.00000,10274.63900,0.00063,9,EU
...,...,...,...,...,...,...,...
57,Tunisia,Kilograms,126299000.00000,68455753.00000,0.54201,11,EU
58,Tunisia,Litres,23520000.00000,15818.00000,0.00067,2,EU
59,Turkey,Kilograms,4045301876.00400,1799497888.22100,0.44484,68,EU
60,Ukraine,Kilograms,6716435736.75000,3154710277.55700,0.46970,77,EU


In [31]:
eu_agg_grouping = eu.groupby(["grouping","quota_unit_final"], as_index = False).agg({"volume":"sum", 
                                                                                  "eu_usage":"sum",
                                                                                  "quota_number":"count"})
eu_agg_grouping["source"] = "EU" 

# rename origin and usage to combine with UK data:

eu_agg_grouping.rename(columns = {'eu_usage':'usage', "volume":"quota_volume"}, inplace = True)

# calculate utilisaiton (fill rate)

eu_agg_grouping["fill_rate"] = eu_agg_grouping["usage"] / eu_agg_grouping["quota_volume"]

col = eu_agg_grouping["fill_rate"]
eu_agg_grouping.drop(labels=["fill_rate"], axis=1, inplace = True)
eu_agg_grouping.insert(4,"fill_rate",col)

eu_agg_grouping.head()

,grouping,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,7,EU
1,Albania,Litres,700000.00000,4038.00000,0.00577,2,EU
2,Andean countries,Kilograms,707581000.00000,84464432.67000,0.11937,77,EU
3,Andean countries,Litre pure (100%) alcohol,390000.00000,0.00000,0.00000,2,EU
4,Andean countries,Litres,28000.00000,0.00000,0.00000,1,EU


In [32]:
eu_agg_region = eu.groupby(["region","quota_unit_final"], as_index = False).agg({"volume":"sum", 
                                                                                  "eu_usage":"sum",
                                                                                  "quota_number":"count"})
eu_agg_region["source"] = "EU" 

# rename origin and usage to combine with UK data:

eu_agg_region.rename(columns = {'eu_usage':'usage', "volume":"quota_volume"}, inplace = True)

# calculate utilisaiton (fill rate)

eu_agg_region["fill_rate"] = eu_agg_region["usage"] / eu_agg_region["quota_volume"]

col = eu_agg_region["fill_rate"]
eu_agg_region.drop(labels=["fill_rate"], axis=1, inplace = True)
eu_agg_region.insert(4,"fill_rate",col)

eu_agg_region.head()

,region,quota_unit_final,quota_volume,usage,fill_rate,quota_number,source
0,Africa,Kilograms,1250952313.00000,701944589.51200,0.56113,58,EU
1,Africa,Litres,137756000.00000,96679497.62500,0.70182,4,EU
2,Asia Pacific,Kilograms,4049103800.09500,1283041327.80100,0.31687,83,EU
3,Asia Pacific,Square metre (m2),17805290.00000,5876166.19700,0.33002,1,EU
4,Eastern Europe,Kilograms,12285437042.75400,5599447746.73800,0.45578,190,EU


**combine aggregated data outputs:**

In [33]:
trq_agg_origin = pd.concat([uk_agg_origin,eu_agg_origin])
print(trq_agg_origin.shape)

# re-order:
trq_agg_origin = trq_agg_origin.sort_values(by=['quota_origin',"quota_unit_final","source"], ascending=True)
trq_agg_origin.head()

(128, 7)


,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,EU,7.00000
0,Albania,Kilograms,384000.00000,0.00000,0.00000,UK,NaN
1,Albania,Litres,700000.00000,4038.00000,0.00577,EU,2.00000
1,Albania,Litres,163400.00000,0.00000,0.00000,UK,NaN
2,Canada,Kilograms,370141000.00000,7833365.20700,0.02116,EU,41.00000


In [34]:
trq_agg_grouping = pd.concat([uk_agg_grouping,eu_agg_grouping])
print(trq_agg_grouping.shape)

# re-order:
trq_agg_grouping = trq_agg_grouping.sort_values(by=['grouping',"quota_unit_final","source"], ascending=True)
trq_agg_grouping

(91, 7)


,grouping,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,EU,7.00000
0,Albania,Kilograms,384000.00000,0.00000,0.00000,UK,NaN
1,Albania,Litres,700000.00000,4038.00000,0.00577,EU,2.00000
1,Albania,Litres,163400.00000,0.00000,0.00000,UK,NaN
2,Andean countries,Kilograms,707581000.00000,84464432.67000,0.11937,EU,77.00000
...,...,...,...,...,...,...,...
43,Turkey,Kilograms,446451000.00000,280871448.63300,0.62912,UK,NaN
43,Ukraine,Kilograms,6716435736.75000,3154710277.55700,0.46970,EU,77.00000
44,Ukraine,Kilograms,417412000.00000,65218829.00000,0.15625,UK,NaN
44,Vietnam,Kilograms,130914973.00000,39727582.00000,0.30346,EU,17.00000


In [35]:
trq_agg_region = pd.concat([uk_agg_region,eu_agg_region])
print(trq_agg_region.shape)

# re-order:
trq_agg_region = trq_agg_region.sort_values(by=['region',"quota_unit_final","source"], ascending=True)
trq_agg_region

(30, 7)


,region,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number
0,Africa,Kilograms,1250952313.00000,701944589.51200,0.56113,EU,58.00000
0,Africa,Kilograms,314629400.00000,185740879.04000,0.59035,UK,NaN
1,Africa,Litres,137756000.00000,96679497.62500,0.70182,EU,4.00000
1,Africa,Litres,72122600.00000,62438545.17600,0.86573,UK,NaN
2,Asia Pacific,Kilograms,4049103800.09500,1283041327.80100,0.31687,EU,83.00000
2,Asia Pacific,Kilograms,237435000.00000,110682361.00000,0.46616,UK,NaN
3,Asia Pacific,Square metre (m2),17805290.00000,5876166.19700,0.33002,EU,1.00000
3,Asia Pacific,Square metre (m2),1068320.00000,0.00000,0.00000,UK,NaN
4,Eastern Europe,Kilograms,12285437042.75400,5599447746.73800,0.45578,EU,190.00000
4,Eastern Europe,Kilograms,904875906.00000,346090277.63300,0.38247,UK,NaN


### 2b. UK quota aggregations using function:

There is duplication of code when writing out the groupby aggregation each time. This can be made more efficient with the use of creating a single function. 

In [36]:
def uk_aggFunc(data, group_type,sector_group = None):
    if(sector_group == None):
        cols = [group_type,"quota_unit_final"]
        final_cols = [group_type,"quota_unit_final","quota_volume","usage","fill_rate","quota_number","source"]
    else:
        cols = [group_type,sector_group,"quota_unit_final"]
        final_cols = [group_type,sector_group,"quota_unit_final","quota_volume","usage","fill_rate","quota_number","source"]
        

    df_agg = data.groupby(cols, as_index = False).agg({"quota_volume":"sum", "remaining_balance":"sum","quota_number":"count"})
    df_agg["source"] = "UK" 
    df_agg["usage"] = df_agg["quota_volume"] - df_agg["remaining_balance"]
    df_agg["fill_rate"] = df_agg["usage"] / df_agg["quota_volume"]
    df_agg = df_agg[final_cols]
    return(df_agg)


In [37]:
df_region = uk_aggFunc(uk3,"region")
df_grouping = uk_aggFunc(uk3,"grouping")
df_origin = uk_aggFunc(uk3,"quota_origin")

print(df_region.shape,df_grouping.shape,df_origin.shape)

(15, 7) (46, 7) (66, 7)


Writing and using a functon to remove repetitive use of similar code is preferable. 

### 3. Compare outputs:

Compare R generated TRQ grouping outputs to python as a quality assurance check:

In [38]:
trq_agg_region2 = pd.read_excel('../rsp_uk_trq_qa/outputs/trq_data_output11.xlsx',"region_level")
trq_agg_grouping2 = pd.read_excel('../rsp_uk_trq_qa/outputs/trq_data_output11.xlsx',"grouping_level")
trq_agg_origin2 = pd.read_excel('../rsp_uk_trq_qa/outputs/trq_data_output11.xlsx',"country_level")
print(trq_agg_origin2.shape,trq_agg_grouping2.shape,trq_agg_region2.shape)

(127, 10) (90, 9) (30, 7)


In [39]:
# compare df shapes:
print(trq_agg_origin.shape,trq_agg_grouping.shape,trq_agg_region.shape)

(128, 7) (91, 7) (30, 7)


In [40]:
# match dfs together. Need joinId first:

trq_agg_region["joinID"] = trq_agg_region["region"]+trq_agg_region["quota_unit_final"]+trq_agg_region["source"]
trq_agg_region2["joinID"] = trq_agg_region2["region"]+trq_agg_region2["quota_unit_final"]+trq_agg_region2["source"]

# join:
trq_agg_region_qa = pd.merge(trq_agg_region,trq_agg_region2[["total_quota_volume",
                                                            "total_quota_usage",
                                                            "total_allocation_fill_rate",
                                                           "joinID"]],on = "joinID", how = "left")
trq_agg_region_qa.shape

(30, 11)

In [41]:
# compare values:

trq_agg_region_qa["volume_check"] = trq_agg_region_qa["quota_volume"]-trq_agg_region_qa["total_quota_volume"]
trq_agg_region_qa["usage_check"] = trq_agg_region_qa["usage"]-trq_agg_region_qa["total_quota_usage"]
trq_agg_region_qa["fill_rate_check"] = trq_agg_region_qa["fill_rate"]-trq_agg_region_qa["total_allocation_fill_rate"]

trq_agg_region_qa

,region,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
0,Africa,Kilograms,1250952313.00000,701944589.51200,0.56113,EU,58.00000,AfricaKilogramsEU,1250952313.00000,701944589.51200,0.56113,0.00000,0.00000,0.00000
1,Africa,Kilograms,314629400.00000,185740879.04000,0.59035,UK,NaN,AfricaKilogramsUK,314629400.00000,186775277.74000,0.59364,0.00000,-1034398.70000,-0.00329
2,Africa,Litres,137756000.00000,96679497.62500,0.70182,EU,4.00000,AfricaLitresEU,137756000.00000,96679497.62500,0.70182,0.00000,0.00000,-0.00000
3,Africa,Litres,72122600.00000,62438545.17600,0.86573,UK,NaN,AfricaLitresUK,72122600.00000,62438545.17600,0.86573,0.00000,0.00000,0.00000
4,Asia Pacific,Kilograms,4049103800.09500,1283041327.80100,0.31687,EU,83.00000,Asia PacificKilogramsEU,4049103800.09500,1283041327.80100,0.31687,0.00000,0.00000,-0.00000
5,Asia Pacific,Kilograms,237435000.00000,110682361.00000,0.46616,UK,NaN,Asia PacificKilogramsUK,221394000.00000,110682361.00000,0.49993,16041000.00000,0.00000,-0.03378
6,Asia Pacific,Square metre (m2),17805290.00000,5876166.19700,0.33002,EU,1.00000,Asia PacificSquare metre (m2)EU,17805290.00000,5876166.19700,0.33002,0.00000,0.00000,-0.00000
7,Asia Pacific,Square metre (m2),1068320.00000,0.00000,0.00000,UK,NaN,Asia PacificSquare metre (m2)UK,1068320.00000,0.00000,0.00000,0.00000,0.00000,0.00000
8,Eastern Europe,Kilograms,12285437042.75400,5599447746.73800,0.45578,EU,190.00000,Eastern EuropeKilogramsEU,12285437042.75400,5599447746.73800,0.45578,0.00000,0.00000,0.00000
9,Eastern Europe,Kilograms,904875906.00000,346090277.63300,0.38247,UK,NaN,Eastern EuropeKilogramsUK,759913906.00000,314154947.63300,0.41341,144962000.00000,31935330.00000,-0.03094


In [42]:
trq_agg_region_qa.dtypes


region                         object
quota_unit_final               object
quota_volume                  float64
usage                         float64
fill_rate                     float64
source                         object
quota_number                  float64
joinID                         object
total_quota_volume            float64
total_quota_usage             float64
total_allocation_fill_rate    float64
volume_check                  float64
usage_check                   float64
fill_rate_check               float64
dtype: object

In [43]:
# country groupings QA:

trq_agg_grouping["joinID"] = trq_agg_grouping["grouping"]+trq_agg_grouping["quota_unit_final"]+trq_agg_grouping["source"]
trq_agg_grouping2["joinID"] = trq_agg_grouping2["grouping"]+trq_agg_grouping2["quota_unit_final"]+trq_agg_grouping2["source"]

# join:
trq_agg_grouping_qa = pd.merge(trq_agg_grouping,trq_agg_grouping2[["total_quota_volume",
                                                            "total_quota_usage",
                                                            "total_allocation_fill_rate",
                                                           "joinID"]],on = "joinID", how = "left")
trq_agg_grouping_qa.shape

(91, 11)

In [44]:
# compare values:

trq_agg_grouping_qa["volume_check"] = trq_agg_grouping_qa["quota_volume"]-trq_agg_grouping_qa["total_quota_volume"]
trq_agg_grouping_qa["usage_check"] = trq_agg_grouping_qa["usage"]-trq_agg_grouping_qa["total_quota_usage"]
trq_agg_grouping_qa["fill_rate_check"] = trq_agg_grouping_qa["fill_rate"]-trq_agg_grouping_qa["total_allocation_fill_rate"]

# str_strip following QA. South Africa has a space at the end of the string:
trq_agg_grouping["grouping"] = trq_agg_grouping["grouping"].str.strip()
trq_agg_grouping_qa.tail(50)

,grouping,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
41,Israel,Litres,121600.00000,121600.00000,1.00000,UK,NaN,IsraelLitresUK,121600.00000,121600.00000,1.00000,0.00000,0.00000,0.00000
42,Israel,Number of items,129920.00000,0.00000,0.00000,EU,1.00000,IsraelNumber of itemsEU,129920.00000,0.00000,0.00000,0.00000,0.00000,0.00000
43,Israel,Number of items,17695.00000,0.00000,0.00000,UK,NaN,IsraelNumber of itemsUK,17695.00000,0.00000,0.00000,0.00000,0.00000,0.00000
44,Japan,Kilograms,79779789.93000,16879143.64100,0.21157,EU,11.00000,JapanKilogramsEU,79779789.93000,16879143.64100,0.21157,-0.00000,0.00000,0.00000
45,Japan,Kilograms,2958000.00000,0.00000,0.00000,UK,NaN,JapanKilogramsUK,NaN,NaN,NaN,NaN,NaN,NaN
46,Jordan,Kilograms,24000000.00000,7568.00000,0.00032,EU,2.00000,JordanKilogramsEU,24000000.00000,7568.00000,0.00032,0.00000,0.00000,0.00000
47,Jordan,Kilograms,2180000.00000,6534.00000,0.00300,UK,NaN,JordanKilogramsUK,2180000.00000,6534.00000,0.00300,0.00000,0.00000,-0.00000
48,Kosovo,Kilograms,35000.00000,0.00000,0.00000,EU,2.00000,KosovoKilogramsEU,35000.00000,0.00000,0.00000,0.00000,0.00000,0.00000
49,Kosovo,Kilograms,70000.00000,0.00000,0.00000,UK,NaN,KosovoKilogramsUK,70000.00000,0.00000,0.00000,0.00000,0.00000,0.00000
50,Kosovo,Litres,5000000.00000,3398564.50000,0.67971,EU,3.00000,KosovoLitresEU,5000000.00000,3398564.50000,0.67971,0.00000,0.00000,0.00000


In [45]:
# country origin QA:


trq_agg_origin["joinID"] = trq_agg_origin["quota_origin"]+trq_agg_origin["quota_unit_final"]+trq_agg_origin["source"]
trq_agg_origin2["joinID"] = trq_agg_origin2["quota_origin"]+trq_agg_origin2["quota_unit_final"]+trq_agg_origin2["source"]

# join:
trq_agg_origin_qa = pd.merge(trq_agg_origin,trq_agg_origin2[["total_quota_volume",
                                                            "total_quota_usage",
                                                            "total_allocation_fill_rate",
                                                           "joinID"]],on = "joinID", how = "left")
trq_agg_origin_qa.shape

(128, 11)

In [46]:
# compare values:

trq_agg_origin_qa["volume_check"] = trq_agg_origin_qa["quota_volume"]-trq_agg_origin_qa["total_quota_volume"]
trq_agg_origin_qa["usage_check"] = trq_agg_origin_qa["usage"]-trq_agg_origin_qa["total_quota_usage"]
trq_agg_origin_qa["fill_rate_check"] = trq_agg_origin_qa["fill_rate"]-trq_agg_origin_qa["total_allocation_fill_rate"]

trq_agg_origin_qa.head()

,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,EU,7.00000,AlbaniaKilogramsEU,2810000.00000,1673986.00000,0.59572,0.00000,0.00000,0.00000
1,Albania,Kilograms,384000.00000,0.00000,0.00000,UK,NaN,AlbaniaKilogramsUK,384000.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,Albania,Litres,700000.00000,4038.00000,0.00577,EU,2.00000,AlbaniaLitresEU,700000.00000,4038.00000,0.00577,0.00000,0.00000,-0.00000
3,Albania,Litres,163400.00000,0.00000,0.00000,UK,NaN,AlbaniaLitresUK,163400.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,Canada,Kilograms,370141000.00000,7833365.20700,0.02116,EU,41.00000,CanadaKilogramsEU,370141000.00000,7833365.20700,0.02116,0.00000,0.00000,-0.00000


### 3b. Create function to upload and QA output files in notebook:

In [47]:
def uk_trq_data_check(sheetName, group):
    # upload data
    sheet_name = sheetName + "_level"
    file_path = '../rsp_uk_trq_qa/outputs/trq_data_output11.xlsx'
    df = pd.read_excel(file_path, sheet_name)
    df["joinID"] = df[group]+df["quota_unit_final"]+df["source"]
    
    # determine QA df and merge:
    if(sheetName == "country"):
        df2 = trq_agg_origin
    elif(sheetName == "grouping"):
        df2 = trq_agg_grouping
    elif(sheetName == "region"):
        df2 = trq_agg_region
               
            
    df2["joinID"] = df2[group]+df2["quota_unit_final"]+df2["source"]
    df3 = pd.merge(df2,df[["total_quota_volume","total_quota_usage","total_allocation_fill_rate","joinID"]], on = "joinID", how = "left")
    
    # check df values:
    
    df3["volume_check"] = round(df3["quota_volume"],0)-round(df3["total_quota_volume"],0)
    df3["usage_check"] = round(df3["usage"],0)-round(df3["total_quota_usage"],0)
    df3["fill_rate_check"] = round(df3["fill_rate"],5)-round(df3["total_allocation_fill_rate"],5)
    
    print(df.shape,df2.shape)
    return(df3)
    

In [48]:
uk_trq_data_check("country","quota_origin")

(127, 11) (128, 8)


,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
0,Albania,Kilograms,2810000.00000,1673986.00000,0.59572,EU,7.00000,AlbaniaKilogramsEU,2810000.00000,1673986.00000,0.59572,0.00000,0.00000,0.00000
1,Albania,Kilograms,384000.00000,0.00000,0.00000,UK,NaN,AlbaniaKilogramsUK,384000.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,Albania,Litres,700000.00000,4038.00000,0.00577,EU,2.00000,AlbaniaLitresEU,700000.00000,4038.00000,0.00577,0.00000,0.00000,0.00000
3,Albania,Litres,163400.00000,0.00000,0.00000,UK,NaN,AlbaniaLitresUK,163400.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,Canada,Kilograms,370141000.00000,7833365.20700,0.02116,EU,41.00000,CanadaKilogramsEU,370141000.00000,7833365.20700,0.02116,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Turkey,Kilograms,446451000.00000,280871448.63300,0.62912,UK,NaN,TurkeyKilogramsUK,348958000.00000,248991938.63300,0.71353,97493000.00000,31879510.00000,-0.08441
124,Ukraine,Kilograms,6716435736.75000,3154710277.55700,0.46970,EU,77.00000,UkraineKilogramsEU,6716435736.75000,3154710277.55700,0.46970,0.00000,0.00000,0.00000
125,Ukraine,Kilograms,417412000.00000,65218829.00000,0.15625,UK,NaN,UkraineKilogramsUK,369943000.00000,65163009.00000,0.17614,47469000.00000,55820.00000,-0.01989
126,Vietnam,Kilograms,130914973.00000,39727582.00000,0.30346,EU,17.00000,VietnamKilogramsEU,130914973.00000,39727582.00000,0.30346,0.00000,0.00000,0.00000


Function practice inserting one function into new one (so one function isn't excessivly large)

In [49]:
def return_diff_outputs(group):
    if(group == "quota_origin"):
        df = uk_trq_data_check("country",group)
    else:
        df = uk_trq_data_check(group,group)
             
    df = df.loc[(df["volume_check"] != 0) | (df["usage_check"] !=0) | (df["fill_rate_check"])]
    print(len(df), " mismatches between outputs and QA code for ",group, " type")
    return(df)


In [50]:
return_diff_outputs("grouping")

(90, 10) (91, 8)
8  mismatches between outputs and QA code for  grouping  type


,grouping,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
29,EEA,Kilograms,35401383.00000,6762040.85900,0.19101,UK,NaN,EEAKilogramsUK,17419383.00000,6757540.85900,0.38793,17982000.00000,4500.00000,-0.19692
45,Japan,Kilograms,2958000.00000,0.00000,0.00000,UK,NaN,JapanKilogramsUK,NaN,NaN,NaN,NaN,NaN,NaN
55,Mexico,Kilograms,193965274.00000,5219659.40000,0.02691,UK,NaN,MexicoKilogramsUK,193965274.00000,5264659.40000,0.02714,0.00000,-45000.00000,-0.00023
59,Morocco,Kilograms,137583000.00000,100372342.84000,0.72954,UK,NaN,MoroccoKilogramsUK,137583000.00000,101406741.54000,0.73706,0.00000,-1034399.00000,-0.00752
67,Serbia,Litres,567526.30000,0.30000,0.00000,UK,NaN,SerbiaLitresUK,56.75263,0.00003,0.00000,567469.00000,0.00000,0.00000
75,South Korea,Kilograms,190403000.00000,97511142.00000,0.51213,UK,NaN,South KoreaKilogramsUK,177320000.00000,97511142.00000,0.54992,13083000.00000,0.00000,-0.03779
86,Turkey,Kilograms,446451000.00000,280871448.63300,0.62912,UK,NaN,TurkeyKilogramsUK,348958000.00000,248991938.63300,0.71353,97493000.00000,31879510.00000,-0.08441
88,Ukraine,Kilograms,417412000.00000,65218829.00000,0.15625,UK,NaN,UkraineKilogramsUK,369943000.00000,65163009.00000,0.17614,47469000.00000,55820.00000,-0.01989


In [51]:
return_diff_outputs("region")

(30, 8) (30, 8)
6  mismatches between outputs and QA code for  region  type


,region,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
1,Africa,Kilograms,314629400.00000,185740879.04000,0.59035,UK,NaN,AfricaKilogramsUK,314629400.00000,186775277.74000,0.59364,0.00000,-1034399.00000,-0.00329
5,Asia Pacific,Kilograms,237435000.00000,110682361.00000,0.46616,UK,NaN,Asia PacificKilogramsUK,221394000.00000,110682361.00000,0.49993,16041000.00000,0.00000,-0.03377
9,Eastern Europe,Kilograms,904875906.00000,346090277.63300,0.38247,UK,NaN,Eastern EuropeKilogramsUK,759913906.00000,314154947.63300,0.41341,144962000.00000,31935330.00000,-0.03094
11,Eastern Europe,Litres,8085726.30000,234283.80000,0.02897,UK,NaN,Eastern EuropeLitresUK,7518256.75263,234283.50003,0.03116,567469.00000,0.00000,-0.00219
13,The Americas,Kilograms,538510273.00000,71164672.66500,0.13215,UK,NaN,The AmericasKilogramsUK,538510273.00000,71209672.66500,0.13223,0.00000,-45000.00000,-0.00008
25,Wider Europe & Middle East,Kilograms,157551652.00000,21094225.21200,0.13389,UK,NaN,Wider Europe & Middle EastKilogramsUK,139569652.00000,21089725.21200,0.15111,17982000.00000,4500.00000,-0.01722


In [52]:
return_diff_outputs("quota_origin")

(127, 11) (128, 8)
11  mismatches between outputs and QA code for  quota_origin  type


,quota_origin,quota_unit_final,quota_volume,usage,fill_rate,source,quota_number,joinID,total_quota_volume,total_quota_usage,total_allocation_fill_rate,volume_check,usage_check,fill_rate_check
60,Japan,Kilograms,2958000.00000,0.00000,0.00000,UK,NaN,JapanKilogramsUK,NaN,NaN,NaN,NaN,NaN,NaN
70,Mexico,Kilograms,193965274.00000,5219659.40000,0.02691,UK,NaN,MexicoKilogramsUK,193965274.00000,5264659.40000,0.02714,0.00000,-45000.00000,-0.00023
74,Morocco,Kilograms,137583000.00000,100372342.84000,0.72954,UK,NaN,MoroccoKilogramsUK,137583000.00000,101406741.54000,0.73706,0.00000,-1034399.00000,-0.00752
77,Nicaragua,Kilograms,73000.00000,0.00000,0.00000,UK,NaN,NicaraguaKilogramsUK,NaN,NaN,NaN,NaN,NaN,NaN
84,Norway,Kilograms,34582918.00000,5968295.73900,0.17258,UK,NaN,NorwayKilogramsUK,16600918.00000,5963795.73900,0.35924,17982000.00000,4500.00000,-0.18666
88,Panama,Kilograms,2027000.00000,0.00000,0.00000,UK,NaN,PanamaKilogramsUK,NaN,NaN,NaN,NaN,NaN,NaN
90,Panama,Litre pure (100%) alcohol,19100.00000,9100.00000,0.47644,UK,NaN,PanamaLitre pure (100%) alcoholUK,NaN,NaN,NaN,NaN,NaN,NaN
104,Serbia,Litres,567526.30000,0.30000,0.00000,UK,NaN,SerbiaLitresUK,56.75263,0.00003,0.00000,567469.00000,0.00000,0.00000
112,South Korea,Kilograms,190403000.00000,97511142.00000,0.51213,UK,NaN,South KoreaKilogramsUK,177320000.00000,97511142.00000,0.54992,13083000.00000,0.00000,-0.03779
123,Turkey,Kilograms,446451000.00000,280871448.63300,0.62912,UK,NaN,TurkeyKilogramsUK,348958000.00000,248991938.63300,0.71353,97493000.00000,31879510.00000,-0.08441


# Section 2. Filled quotas QA

Filled quotas are where the fill_rate is > 0.999%. These quotas have a last allocation data when the quota was last used. This field can be compared against the quota start period to produce an estimated time the quota was filled. This is an important metric in this data set. 

The data fileds need converting. Dataset needs filtering then the final comparison needs calculating for this metric calculation. 

In [53]:
uk_trqs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   quota_number               1067 non-null   object 
 1   quota_product_description  1067 non-null   object 
 2   year                       1067 non-null   int64  
 3   quota_origin               1067 non-null   object 
 4   validity_start_date        1067 non-null   object 
 5   validity_end_date          1067 non-null   object 
 6   quota_volume               1067 non-null   float64
 7   remaining_balance          1067 non-null   float64
 8   quota_fill_rate            1066 non-null   float64
 9   quota_unit                 1067 non-null   object 
 10  commodity_codes            1067 non-null   object 
 11  quota_status               1067 non-null   object 
 12  last_allocation_date       408 non-null    object 
 13  trq_type                   1067 non-null   objec

In [54]:
# convert to datatime
uk_trqs["validity_start_date"]  = pd.to_datetime(uk_trqs["validity_start_date"],format = "%d/%m/%Y")
uk_trqs["validity_end_date"]    = pd.to_datetime(uk_trqs["validity_end_date"],format = "%d/%m/%Y")
uk_trqs["last_allocation_date"] = pd.to_datetime(uk_trqs["last_allocation_date"],format = "%d/%m/%Y")

In [55]:
# filter data
uk_filled = uk_trqs.loc[uk_trqs["quota_fill_rate"] >= 0.999].copy()
uk_filled.shape

(69, 14)

In [56]:
uk_filled.isnull().sum() # highlight if NAs:

quota_number                 0
quota_product_description    0
year                         0
quota_origin                 0
validity_start_date          0
validity_end_date            0
quota_volume                 0
remaining_balance            0
quota_fill_rate              0
quota_unit                   0
commodity_codes              0
quota_status                 0
last_allocation_date         2
trq_type                     0
dtype: int64

In [57]:
nas = uk_filled[uk_filled["last_allocation_date"].isnull()]

In [58]:
uk_filled["last_days"] = uk_filled["last_allocation_date"]-uk_filled["validity_start_date"]
uk_filled["no_days"] = uk_filled["validity_end_date"] - uk_filled["validity_start_date"]
uk_filled = uk_filled[~(uk_filled["last_allocation_date"].isnull())] # filter away NaN.
uk_filled = uk_filled.loc[uk_filled["last_allocation_date"] >= "01/01/2021"] # remove last allocaiton dates which are incorrect in data (i.e. < than 2021)
uk_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 4 to 1020
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype          
---  ------                     --------------  -----          
 0   quota_number               65 non-null     object         
 1   quota_product_description  65 non-null     object         
 2   year                       65 non-null     int64          
 3   quota_origin               65 non-null     object         
 4   validity_start_date        65 non-null     datetime64[ns] 
 5   validity_end_date          65 non-null     datetime64[ns] 
 6   quota_volume               65 non-null     float64        
 7   remaining_balance          65 non-null     float64        
 8   quota_fill_rate            65 non-null     float64        
 9   quota_unit                 65 non-null     object         
 10  commodity_codes            65 non-null     object         
 11  quota_status               65 non-null     object         

group by quota origin for all filled quotas and calculate average quota length and time for quota to be filled:

In [59]:
def filledFunc(data,extraGroup = None):
    if(extraGroup == None):
        cols = ["quota_origin"]
    else:
        cols = ["quota_origin",extraGroup] # defined extra grouping for sector data later. 
    
    df = data.copy()
    df[["last_days","no_days"]] = df[["last_days","no_days"]].astype('timedelta64[D]') # convert data output to numeric
    df_agg = df.groupby(cols, as_index = False).agg({"quota_number":"count","last_days":"mean","no_days":"mean"})
    return(df_agg)

In [60]:
uk_filled_agg = filledFunc(uk_filled)
uk_filled_agg

,quota_origin,quota_number,last_days,no_days
0,Countries subject to UK safeguard measures,9,48.55556,91.66667
1,ERGA OMNES,12,177.83333,320.25000
2,Egypt,2,75.00000,166.00000
3,European Union,4,57.75000,90.75000
4,Faroe Islands,1,54.00000,365.00000
5,Iceland,3,132.66667,365.00000
6,India,1,56.00000,92.00000
7,Israel,2,135.50000,365.00000
8,Mexico,1,24.00000,275.00000
9,Morocco,4,91.75000,181.50000


pivot data into long format for QA of TRQ outputs:

In [61]:
uk_filled_long = uk_filled_agg.melt(id_vars = ["quota_origin","quota_number"])
uk_filled_long = uk_filled_long.sort_values(by=['quota_origin'], ascending=True)
uk_filled_long

,quota_origin,quota_number,variable,value
0,Countries subject to UK safeguard measures,9,last_days,48.55556
17,Countries subject to UK safeguard measures,9,no_days,91.66667
1,ERGA OMNES,12,last_days,177.83333
18,ERGA OMNES,12,no_days,320.25000
19,Egypt,2,no_days,166.00000
2,Egypt,2,last_days,75.00000
3,European Union,4,last_days,57.75000
20,European Union,4,no_days,90.75000
21,Faroe Islands,1,no_days,365.00000
4,Faroe Islands,1,last_days,54.00000


In [62]:
# cant sue wide_to_long as stubnames has no common start string in fields. 
#uk_filled_long = pd.wide_to_long(df, stubnames ="days", i = ["quota_origin","quota_number"], j="cat")

Upload outputs for QA and comparison:

Function created so not to repeat lines when sector data is required to run through same process:

In [63]:
# function:
def filled_quotas_check(data, sectorSelect = None):
    # NOTE for sector data - more lines to be added: tbc. 
    print("Fill UK QUOTA CHECK... return df of none matches:")
    df = pd.read_excel('../rsp_uk_trq_qa/outputs/uk_trqs_filled10.xlsx')
    # change df labels to match QA df:
    conditions = [(df["quota_length_cat"] == "avg_quota_fill"), (df["quota_length_cat"] == "avg_quota_length")]
    choices = ["last_days","no_days"]
    df['cat'] = np.select(conditions, choices, default=np.nan)
    # create unique identifier for merge:
    df["joinID"]=df["quota_origin"]+df["cat"]
    data["joinID"]=data["quota_origin"]+data["variable"]
    dfm = pd.merge(data, df, on = "joinID", how = "left")
    # filter matches:
    dfm = dfm.loc[~(dfm["cat"].isnull())]
    # qa value check:
    dfm["value_qa"] = round(dfm["value"],1)-round(dfm["quota_length_val"],1)
    dfqa = dfm.loc[dfm["value_qa"] > 0] 
    return(dfqa)

In [64]:
filled_quotas_check(uk_filled_long)

Fill UK QUOTA CHECK... return df of none matches:


,quota_origin_x,quota_number,variable,value,joinID,quota_origin_y,grouping,region,quota_count,total_quota_volume,quota_length_cat,quota_length_val,cat,value_qa
19,Morocco,4,last_days,91.75000,Moroccolast_days,Morocco,Morocco,Africa,4.00000,35208000.00000,avg_quota_fill,91.72917,last_days,0.10000
21,Norway,7,last_days,109.85714,Norwaylast_days,Norway,EEA,Wider Europe & Middle East,7.00000,4826000.00000,avg_quota_fill,109.83929,last_days,0.10000
28,Turkey,9,no_days,214.66667,Turkeyno_days,Turkey,Turkey,Eastern Europe,10.00000,118264000.00000,avg_quota_length,202.20417,no_days,12.50000


#### Sector Data compilation:

Sector data needs creating automating the process utilising the commodity code strings. this is to save time going through 700+ rows manually assigned labels. 

Unconcatonate each commodity code grouped by quota into a single row, then match in the hs_descriptions and analysis summarises the most common HS2 (two digit) chapter for quota sector

In [65]:
# NOTE: uk3 is final uk trq df following cleaning and combined all uk quota data inputs.
dfs = uk3[["quota_number","commodity_codes"]]

In [66]:
new_df = pd.DataFrame(dfs.commodity_codes.str.split(';').tolist(), index=dfs.quota_number).stack()
new_df = new_df.reset_index([0, 'quota_number'])
new_df.columns = ['quota_number', 'commodity_codes']
new_df[ 'commodity_codes'] = new_df[ 'commodity_codes'].str.strip() # remove whitespace
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4933 entries, 0 to 4932
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   quota_number     4933 non-null   object
 1   commodity_codes  4933 non-null   object
dtypes: object(2)
memory usage: 77.2+ KB


In [67]:
# strip HS2 codes:
new_df["hs2"] = new_df["commodity_codes"].str[:2]
hs_dsc["hs2"] = hs_dsc["hs2_description"].str[:2]

In [68]:
dfs.head()

,quota_number,commodity_codes
0,050146,0102294100 ; 0102294900 ; 0102295100 ; 0102295...
1,050155,0207423000 ; 0207428000 ; 0207441000 ; 0207442...
2,050202,0701900000
3,050204,0806101090
4,050204,0806101090


Next need to aggregated upto quota/hs2 level the counts of each HS2 chapter. 

In [69]:
dfs_agg = new_df.groupby(["quota_number","hs2"], as_index = False).agg({"commodity_codes":"count"})
dfs_agg["ncount"] = dfs_agg.groupby("quota_number",as_index = False)["hs2"].transform('count')
# match hs_dsc
dfs_agg = pd.merge(dfs_agg,hs_dsc,on ="hs2",how="left")

In [70]:
# calculate the proportion of each HS chapter across a quota number
dfs_agg["codes_count"]=dfs_agg.groupby("quota_number",as_index = False)["commodity_codes"].transform("sum")
dfs_agg["hs2_prop"]=dfs_agg["commodity_codes"]/dfs_agg["codes_count"]

In [71]:
dfs_final = dfs_agg

In [72]:
dfs_agg["max_count"]=dfs_agg.groupby("quota_number",as_index = False)["commodity_codes"].transform("max")
dfs_agg

,quota_number,hs2,commodity_codes,ncount,hs_section,rsp_hs_section,hs2_description,codes_count,hs2_prop,max_count
0,050146,01,10,1,01 - Animal products,Live animals,01 - LIVE ANIMALS,10,1.00000,10
1,050155,02,46,1,01 - Animal products,Animal products,02 - MEAT AND EDIBLE MEAT OFFAL,46,1.00000,46
2,050202,07,1,1,02 - Vegetable products,Fruits and vegetables,07 - EDIBLE VEGETABLES AND CERTAIN ROOTS AND T...,1,1.00000,1
3,050204,08,2,1,02 - Vegetable products,Fruits and vegetables,08 - EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRU...,2,1.00000,2
4,050206,15,4,1,03 - Fats and oils,Fats and oils,15 - ANIMAL OR VEGETABLE FATS AND OILS AND THE...,4,1.00000,4
...,...,...,...,...,...,...,...,...,...,...
746,059228,04,1,1,01 - Animal products,"Dairy, eggs, honey",04 - DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY...,1,1.00000,1
747,059229,35,2,1,06 - Chemical products,Chemical products,35 - ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHE...,2,1.00000,2
748,059280,02,19,1,01 - Animal products,Animal products,02 - MEAT AND EDIBLE MEAT OFFAL,19,1.00000,19
749,059281,02,47,1,01 - Animal products,Animal products,02 - MEAT AND EDIBLE MEAT OFFAL,47,1.00000,47


In [73]:
dfs_final = dfs_agg.loc[dfs_agg["commodity_codes"] == dfs_agg["max_count"]].copy()
dfs_final["hs_section"]=dfs_final["hs_section"].str.replace('\d+', '').str.replace("-","") #remove numerical values and "-"
dfs_final["sector_dsc"] = np.where(dfs_final["hs2_prop"] > 0.5,dfs_final["rsp_hs_section"],dfs_final["hs_section"])
dfs_final

<ipython-input-73-0e13eea71c8d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dfs_final["hs_section"]=dfs_final["hs_section"].str.replace('\d+', '').str.replace("-","")


,quota_number,hs2,commodity_codes,ncount,hs_section,rsp_hs_section,hs2_description,codes_count,hs2_prop,max_count,sector_dsc
0,050146,01,10,1,Animal products,Live animals,01 - LIVE ANIMALS,10,1.00000,10,Live animals
1,050155,02,46,1,Animal products,Animal products,02 - MEAT AND EDIBLE MEAT OFFAL,46,1.00000,46,Animal products
2,050202,07,1,1,Vegetable products,Fruits and vegetables,07 - EDIBLE VEGETABLES AND CERTAIN ROOTS AND T...,1,1.00000,1,Fruits and vegetables
3,050204,08,2,1,Vegetable products,Fruits and vegetables,08 - EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRU...,2,1.00000,2,Fruits and vegetables
4,050206,15,4,1,Fats and oils,Fats and oils,15 - ANIMAL OR VEGETABLE FATS AND OILS AND THE...,4,1.00000,4,Fats and oils
...,...,...,...,...,...,...,...,...,...,...,...
746,059228,04,1,1,Animal products,"Dairy, eggs, honey",04 - DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY...,1,1.00000,1,"Dairy, eggs, honey"
747,059229,35,2,1,Chemical products,Chemical products,35 - ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHE...,2,1.00000,2,Chemical products
748,059280,02,19,1,Animal products,Animal products,02 - MEAT AND EDIBLE MEAT OFFAL,19,1.00000,19,Animal products
749,059281,02,47,1,Animal products,Animal products,02 - MEAT AND EDIBLE MEAT OFFAL,47,1.00000,47,Animal products


In [74]:
# match final sector quota classification back to main dfs df:

In [91]:
uks = pd.merge(uk3,dfs_final[["quota_number","sector_dsc"]],on="quota_number",how="left")
print(uks.shape,uk3.shape)

(810, 14) (810, 13)


df length is different - which means there are multiple rows for select quota numbers. Need to remove duplicates:

In [94]:
dfs_final = dfs_final.drop_duplicates(subset=['quota_number'])
uks = pd.merge(uk3,dfs_final[["quota_number","sector_dsc"]],on="quota_number",how="left")
print(uks.shape,uk3.shape)

(810, 14) (810, 13)


In [97]:
#relocate sector in df:
col = uks[["sector_dsc"]]
uks.drop(labels=["sector_dsc"], axis=1, inplace = True)
uks.insert(2,"sector_dsc",col)
uks.head()

,quota_number,quota_description,sector_dsc,quota_year,quota_origin,quota_volume,remaining_balance,quota_unit,commodity_codes,iso,country_name,grouping,region,quota_unit_final
0,050146,Live bovine animals,Live animals,2021,Switzerland,247.00000,247.00000,Number of items (p/st),0102294100 ; 0102294900 ; 0102295100 ; 0102295...,CH,Switzerland,Switzerland,Wider Europe & Middle East,Number of items
1,050155,"Meat and edible offal, of duch, fresh or forzen",Animal products,2021,Israel,76000.00000,76000.00000,Kilogram (kg),0207423000 ; 0207428000 ; 0207441000 ; 0207442...,IL,Israel,Israel,Wider Europe & Middle East,Kilograms
2,050202,"Potatoes, fresh or chilled, other than seed",Fruits and vegetables,2021,Turkey,417000.00000,322220.00000,Kilogram (kg),0701900000,TR,Turkey,Turkey,Eastern Europe,Kilograms
3,050204,Fresh table grapes,Fruits and vegetables,2021,Turkey,58000.00000,58000.00000,Kilogram (kg),0806101090,TR,Turkey,Turkey,Eastern Europe,Kilograms
4,050204,Fresh table grapes,Fruits and vegetables,2021,Turkey,58000.00000,0.00000,Kilogram (kg),0806101090,TR,Turkey,Turkey,Eastern Europe,Kilograms


# Section 3: Sector quota data QA:

#### Now sector data is assigned to each quota - can produce sector level outptus for QA checks:

In [105]:
trq_sector_agg_region = uk_aggFunc(uks,"region",sector_group="sector_dsc")
trq_sector_agg_grouping = uk_aggFunc(uks,"grouping",sector_group="sector_dsc")
trq_sector_agg_origin = uk_aggFunc(uks,"quota_origin",sector_group="sector_dsc")
print(trq_sector_agg_region.info(),trq_sector_agg_grouping.info(),trq_sector_agg_origin.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 0 to 77
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   region            78 non-null     object 
 1   sector_dsc        78 non-null     object 
 2   quota_unit_final  78 non-null     object 
 3   quota_volume      78 non-null     float64
 4   usage             78 non-null     float64
 5   fill_rate         78 non-null     float64
 6   quota_number      78 non-null     int64  
 7   source            78 non-null     object 
dtypes: float64(3), int64(1), object(4)
memory usage: 5.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 160 entries, 0 to 159
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   grouping          160 non-null    object 
 1   sector_dsc        160 non-null    object 
 2   quota_unit_final  160 non-null    object 
 3   quota_volume      160 

In [ ]:
# create QA function for sector 

In [112]:
def uk_trq_sector_data_check(sheetName, group):
    # upload data
    sheet_name = sheetName + "_level"
    file_path = '../rsp_uk_trq_qa/outputs/trq_sector_data_output10.xlsx'
    df = pd.read_excel(file_path, sheet_name)
    df["joinID"] = df[group]+df["quota_unit_final"]+df["source"]+df["trq_dsc"]
    # remove EU data for this check:
    df=df.loc[df["source"]=="UK"]
    # determine QA df and merge:
    if(sheetName == "country"):
        df2 = trq_sector_agg_origin
    elif(sheetName == "grouping"):
        df2 = trq_sector_agg_grouping
    elif(sheetName == "region"):
        df2 = trq_sector_agg_region
               
            
    df2["joinID"] = df2[group]+df2["quota_unit_final"]+df2["source"]+df2["sector_dsc"]
    df3 = pd.merge(df2,df[["total_quota_volume","total_quota_usage","total_allocation_fill_rate","joinID"]], on = "joinID", how = "left")
    
    # check df values:
    
    df3["volume_check"] = round(df3["quota_volume"],0)-round(df3["total_quota_volume"],0)
    df3["usage_check"] = round(df3["usage"],0)-round(df3["total_quota_usage"],0)
    df3["fill_rate_check"] = round(df3["fill_rate"],5)-round(df3["total_allocation_fill_rate"],5)
    
    print(df.shape,df2.shape)
    return(df3)

In [118]:
#after initial check - sector names have whitespaces which need removing
def removeWS(df):
    df["sector_dsc"]=df["sector_dsc"].str.strip()
    return(df)

trq_sector_agg_origin= removeWS(trq_sector_agg_origin)
trq_sector_agg_grouping= removeWS(trq_sector_agg_grouping)
trq_sector_agg_region= removeWS(trq_sector_agg_region)

In [119]:
region_sector_check = uk_trq_sector_data_check("region",group="region")
grouping_sector_check = uk_trq_sector_data_check("grouping",group="grouping")
origin_sector_check = uk_trq_sector_data_check("country",group="quota_origin")

(74, 9) (78, 9)
(155, 11) (160, 9)
(188, 12) (193, 9)


In [120]:
trq_sector_agg_region = uk_aggFunc(uks,"region",sector_group="sector_dsc")
trq_sector_agg_grouping = uk_aggFunc(uks,"grouping",sector_group="sector_dsc")
trq_sector_agg_origin = uk_aggFunc(uks,"quota_origin",sector_group="sector_dsc")

In [121]:
# save sector output files:
with pd.ExcelWriter('../rsp_uk_trq_qa/outputs/trq_sector_data_output_qa.xlsx') as writer1:
    region_sector_check.to_excel(writer1, sheet_name = 'region', index = False)
    grouping_sector_check.to_excel(writer1, sheet_name = "grouping", index = False)
    origin_sector_check.to_excel(writer1, sheet_name = 'quota_origin', index = False)    

End.